# Question Generation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepset-ai/haystack-tutorials/blob/main/tutorials/13_Question_generation.ipynb)

This is a bare bones tutorial showing what is possible with the QuestionGenerator Nodes and Pipelines which automatically
generate questions which the question generation model thinks can be answered by a given document.

### Prepare environment

#### Colab: Enable the GPU runtime
Make sure you enable the GPU runtime to experience decent speed in this tutorial.  
**Runtime -> Change Runtime type -> Hardware accelerator -> GPU**

<img src="https://raw.githubusercontent.com/deepset-ai/haystack/main/docs/img/colab_gpu_runtime.jpg">

In [1]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest main of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 30.9 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-i7fupa1l/farm-haystack_22010227bdf3487888c22a63884f04a4
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-i7fupa1l/farm-haystack_22010227bdf3487888c22a63884f04a4
  Resolved https://github.com/deepset-ai/haystack.git to commit b84a6b17165dbf10665794b5decdefa113300749
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.4/381.4 kB 25

## Logging

We configure how logging messages should be displayed and which log level should be used before importing Haystack.
Example log message:
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/218_Olenna_Tyrell.txt
Default log level in basicConfig is WARNING so the explicit parameter is not necessary but can be changed easily:

In [2]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [3]:
# Imports needed to run this notebook

from pprint import pprint
from tqdm import tqdm
from haystack.nodes import QuestionGenerator, BM25Retriever, FARMReader
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.pipelines import (
    QuestionGenerationPipeline,
    RetrieverQuestionGenerationPipeline,
    QuestionAnswerGenerationPipeline,
)
from haystack.utils import launch_es, print_questions

Let's start an Elasticsearch instance with one of the options below:

In [4]:
# Option 1: Start Elasticsearch service via Docker
launch_es()

In [5]:
# Option 2: In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

Let's initialize some core components

In [6]:
text1 = "Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable use of significant whitespace."
text2 = "Princess Arya Stark is the third child and second daughter of Lord Eddard Stark and his wife, Lady Catelyn Stark. She is the sister of the incumbent Westerosi monarchs, Sansa, Queen in the North, and Brandon, King of the Andals and the First Men. After narrowly escaping the persecution of House Stark by House Lannister, Arya is trained as a Faceless Man at the House of Black and White in Braavos, using her abilities to avenge her family. Upon her return to Westeros, she exacts retribution for the Red Wedding by exterminating the Frey male line."
text3 = "Dry Cleaning are an English post-punk band who formed in South London in 2018.[3] The band is composed of vocalist Florence Shaw, guitarist Tom Dowse, bassist Lewis Maynard and drummer Nick Buxton. They are noted for their use of spoken word primarily in lieu of sung vocals, as well as their unconventional lyrics. Their musical stylings have been compared to Wire, Magazine and Joy Division.[4] The band released their debut single, 'Magic of Meghan' in 2019. Shaw wrote the song after going through a break-up and moving out of her former partner's apartment the same day that Meghan Markle and Prince Harry announced they were engaged.[5] This was followed by the release of two EPs that year: Sweet Princess in August and Boundary Road Snacks and Drinks in October. The band were included as part of the NME 100 of 2020,[6] as well as DIY magazine's Class of 2020.[7] The band signed to 4AD in late 2020 and shared a new single, 'Scratchcard Lanyard'.[8] In February 2021, the band shared details of their debut studio album, New Long Leg. They also shared the single 'Strong Feelings'.[9] The album, which was produced by John Parish, was released on 2 April 2021.[10]"

docs = [{"content": text1}, {"content": text2}, {"content": text3}]

# Initialize document store and write in the documents
document_store = ElasticsearchDocumentStore()
document_store.delete_documents()
document_store.write_documents(docs)

# Initialize Question Generator
question_generator = QuestionGenerator()

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Using sep_token, but it is not set yet.


## Question Generation Pipeline

The most basic version of a question generator pipeline takes a document as input and outputs generated questions
which the the document can answer.

In [7]:
question_generation_pipeline = QuestionGenerationPipeline(question_generator)
for idx, document in enumerate(document_store):

    print(f"\n * Generating questions for document {idx}: {document.content[:100]}...\n")
    result_a = question_generation_pipeline.run(documents=[document])
    print_questions(result_a)


 * Generating questions for document 0: Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Ros...


Generated questions:
 - Who created Python?
 - When was Python first released?
 - What is Python's design philosophy?

 * Generating questions for document 1: Princess Arya Stark is the third child and second daughter of Lord Eddard Stark and his wife, Lady C...


Generated questions:
 - Who is the third child and second daughter of Lord Eddard Stark and his wife, Lady Catelyn Stark?
 - Princess Arya Stark is the sister of what Westerosi monarchs?
 - What is Sansa, Queen in the North, and Brandon, King of the Andals?
 - What is Arya trained as?
 - Where is the House of Black and White located?
 - What is the name of the first men?
 - What is the name of the line that Frey exterminates?
 - Where does the Red Wedding take place?

 * Generating questions for document 2: Dry Cleaning are an English post-punk band who formed in South London in 20

## Retriever Question Generation Pipeline

This pipeline takes a query as input. It retrieves relevant documents and then generates questions based on these.

In [8]:
retriever = BM25Retriever(document_store=document_store)
rqg_pipeline = RetrieverQuestionGenerationPipeline(retriever, question_generator)

print(f"\n * Generating questions for documents matching the query 'Arya Stark'\n")
result_b = rqg_pipeline.run(query="Arya Stark")
print_questions(result_b)


 * Generating questions for documents matching the query 'Arya Stark'


Generated questions:
 - Who is the third child and second daughter of Lord Eddard Stark and his wife, Lady Catelyn Stark?
 - Princess Arya Stark is the sister of what Westerosi monarchs?
 - What is Sansa, Queen in the North, and Brandon, King of the Andals?
 - What is Arya trained as?
 - Where is the House of Black and White located?
 - What is the name of the first men?
 - What is the name of the line that Frey exterminates?
 - Where does the Red Wedding take place?


## Question Answer Generation Pipeline

This pipeline takes a document as input, generates questions on it, and attempts to answer these questions using
a Reader model

In [9]:
reader = FARMReader("deepset/roberta-base-squad2")
results = []
qag_pipeline = QuestionAnswerGenerationPipeline(question_generator, reader)
for idx, document in enumerate(tqdm(document_store)):

    print(f"\n * Generating questions and answers for document {idx}: {document.content[:100]}...\n")
    result_c = qag_pipeline.run(documents=[document])
    results.append(result_c)
    print_questions(result_c)


INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
0it [00:00, ?it/s]


 * Generating questions and answers for document 0: Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Ros...



...


Generated pairs:
 - Q: Who created Python?
      A: Guido van Rossum
 - Q: When was Python first released?
      A: 1991
 - Q: What is Python's design philosophy?
      A: emphasizes code readability

 * Generating questions and answers for document 1: Princess Arya Stark is the third child and second daughter of Lord Eddard Stark and his wife, Lady C...




Inferencing Samples: 100%|██████████| 1/1 [00:13<00:00, 13.97s/ Batches]
2it [00:42, 23.56s/it]


Generated pairs:
 - Q: Who is the third child and second daughter of Lord Eddard Stark and his wife, Lady Catelyn Stark?
      A: Princess Arya Stark
 - Q: Princess Arya Stark is the sister of what Westerosi monarchs?
      A: Sansa, Queen in the North, and Brandon, King of the Andals and the First Men
 - Q: What is Sansa, Queen in the North, and Brandon, King of the Andals?
      A: the sister
 - Q: What is Arya trained as?
      A: Faceless Man
 - Q: Where is the House of Black and White located?
      A: Braavos
 - Q: What is the name of the first men?
      A: Brandon
 - Q: What is the name of the line that Frey exterminates?
      A: Frey male line
 - Q: Where does the Red Wedding take place?
      A: Westeros

 * Generating questions and answers for document 2: Dry Cleaning are an English post-punk band who formed in South London in 2018.[3] The band is compos...




Inferencing Samples: 100%|██████████| 1/1 [01:00<00:00, 60.09s/ Batches]
3it [02:07, 42.49s/it]


Generated pairs:
 - Q: What is the name of the English post-punk band that formed in South London in 2018?
      A: Dry Cleaning
      A: Boundary Road Snacks and Drinks
 - Q: Who is the vocalist of Dry Cleaning?
      A: Florence Shaw
      A: ghan Markle and Prince Harry announced they were engaged.[5] This was followed by the release of two EPs that year: Sweet Princess in August and Boundary Road Snacks and Drinks in October. The band were included as part of the NME 100 of 2020,[6] as well as DIY magazine's Class of 2020.[7] The band signed to 4AD in late 2020 and shared a new single, 'Scratchcard Lanyard'.[8] In February 2021, the band shared details of their debut studio album, New Long Leg. They also shared the single 'Strong Feelings'.[9] The album, which was produced by John Parish
 - Q: Where did Dry Cleaning form?
      A: South London
      A: Boundary Road
 - Q: What does the band use instead of sung vocals?
      A: spoken word
      A: 2020,[6] as well as DIY magazine'

In [10]:
results

[{'queries': ['Who created Python?',
   'When was Python first released?',
   "What is Python's design philosophy?"],
  'answers': [[<Answer {'answer': 'Guido van Rossum', 'type': 'extractive', 'score': 0.9956661462783813, 'context': "eted, high-level, general-purpose programming language. Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes ", 'offsets_in_document': [{'start': 87, 'end': 103}], 'offsets_in_context': [{'start': 67, 'end': 83}], 'document_id': '2ce1de1b4d6dd8e4564795c955e0b356', 'meta': {}}>],
   [<Answer {'answer': '1991', 'type': 'extractive', 'score': 0.992423415184021, 'context': " programming language. Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable", 'offsets_in_document': [{'start': 126, 'end': 130}], 'offsets_in_context': [{'start': 73, 'end': 77}], 'document_id': '2ce1de1b4d6dd8e4564795c955e0b356', 'meta': {}}>],
   [<Answer {'answer': 'empha

In [11]:
q = []
a = []
c = []
q__ = []
for i in range (3):
  q.append(results[i]["queries"])
  for j in range(3):
    aa=results[i]["answers"][j][0].answer
    a.append(aa)
    cc=results[i]["documents"][j][0].content
    c.append(cc)
q__.append({"Question":q,"Answer Generation Pipeline":a, "Comtext":c})

In [12]:
q__

[{'Question': [['Who created Python?',
    'When was Python first released?',
    "What is Python's design philosophy?"],
   ['Who is the third child and second daughter of Lord Eddard Stark and his wife, Lady Catelyn Stark?',
    'Princess Arya Stark is the sister of what Westerosi monarchs?',
    'What is Sansa, Queen in the North, and Brandon, King of the Andals?',
    'What is Arya trained as?',
    'Where is the House of Black and White located?',
    'What is the name of the first men?',
    'What is the name of the line that Frey exterminates?',
    'Where does the Red Wedding take place?'],
   ['What is the name of the English post-punk band that formed in South London in 2018?',
    'Who is the vocalist of Dry Cleaning?',
    'Where did Dry Cleaning form?',
    'What does the band use instead of sung vocals?',
    "What is the name of the band's debut single?",
    "What was Shaw's first song called?",
    'When did Shaw write the song Magic of Meghan?',
    'When did Meghan M

In [13]:
q__[0]['Question']

[['Who created Python?',
  'When was Python first released?',
  "What is Python's design philosophy?"],
 ['Who is the third child and second daughter of Lord Eddard Stark and his wife, Lady Catelyn Stark?',
  'Princess Arya Stark is the sister of what Westerosi monarchs?',
  'What is Sansa, Queen in the North, and Brandon, King of the Andals?',
  'What is Arya trained as?',
  'Where is the House of Black and White located?',
  'What is the name of the first men?',
  'What is the name of the line that Frey exterminates?',
  'Where does the Red Wedding take place?'],
 ['What is the name of the English post-punk band that formed in South London in 2018?',
  'Who is the vocalist of Dry Cleaning?',
  'Where did Dry Cleaning form?',
  'What does the band use instead of sung vocals?',
  "What is the name of the band's debut single?",
  "What was Shaw's first song called?",
  'When did Shaw write the song Magic of Meghan?',
  'When did Meghan Markle and Prince Harry announce they were engaged?

In [14]:
q__[0]['Answer Generation Pipeline']

['Guido van Rossum',
 '1991',
 'emphasizes code readability',
 'Princess Arya Stark',
 'Sansa, Queen in the North, and Brandon, King of the Andals and the First Men',
 'the sister',
 'Dry Cleaning',
 'Florence Shaw',
 'South London']

In [15]:
q__[0]['Comtext']

["Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable use of significant whitespace.",
 "Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable use of significant whitespace.",
 "Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable use of significant whitespace.",
 'Princess Arya Stark is the third child and second daughter of Lord Eddard Stark and his wife, Lady Catelyn Stark. She is the sister of the incumbent Westerosi monarchs, Sansa, Queen in the North, and Brandon, King of the Andals and the First Men. After narrowly escaping the persec

In [16]:
q_a_c = []
for list_ in results:
  for q , a, c in zip(list_['queries'], list_['answers'], list_['answers']):
    a = a[0]
    c = c[0]
    q_a_c.append({"Question":q, "Answer Generation Pipeline":a.answer, "Comtext":c.context})

In [17]:
q_a_c

[{'Question': 'Who created Python?',
  'Answer Generation Pipeline': 'Guido van Rossum',
  'Comtext': "eted, high-level, general-purpose programming language. Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes "},
 {'Question': 'When was Python first released?',
  'Answer Generation Pipeline': '1991',
  'Comtext': " programming language. Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable"},
 {'Question': "What is Python's design philosophy?",
  'Answer Generation Pipeline': 'emphasizes code readability',
  'Comtext': "by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable use of significant whitespace."},
 {'Question': 'Who is the third child and second daughter of Lord Eddard Stark and his wife, Lady Catelyn Stark?',
  'Answer Generation Pipeline': 'Princess Arya Stark',
  'Comtext': 'Princess Arya Star

In [37]:
q_a_c[1]['Question']

'When was Python first released?'

In [38]:
len(q_a_c)

28

In [18]:
results[0]["queries"]

['Who created Python?',
 'When was Python first released?',
 "What is Python's design philosophy?"]

In [19]:
len(results[0]["queries"])

3

##**write to file csv**

In [42]:
Question_Generation_Pipeline = []
Context_Pipeline = []
Answer_Generation_Pipeline = []
for i in range(len(q_a_c)):
  Question_Generation_Pipeline.append(q_a_c[i]['Question'])
  Context_Pipeline.append(q_a_c[i]['Comtext'])
  Answer_Generation_Pipeline.append(q_a_c[i]['Answer Generation Pipeline']) 


In [46]:
Context_Pipeline

["eted, high-level, general-purpose programming language. Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes ",
 " programming language. Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable",
 "by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable use of significant whitespace.",
 'Princess Arya Stark is the third child and second daughter of Lord Eddard Stark and his wife, Lady Catelyn Stark. She is the sister of the incumbent W',
 'of the incumbent Westerosi monarchs, Sansa, Queen in the North, and Brandon, King of the Andals and the First Men. After narrowly escaping the persecu',
 'aughter of Lord Eddard Stark and his wife, Lady Catelyn Stark. She is the sister of the incumbent Westerosi monarchs, Sansa, Queen in the North, and B',
 ' persecution of House Stark by House Lannister, Arya is trained as a

In [53]:
import csv 
    
# field names 
fields = ["Context_Pipeline","Question_Generation_Pipeline", "Answer_Generation_Pipeline"]
rows = []    
i = 0
# data rows of csv file 
for i in range(len(q_a_c)):
  rows.append([Context_Pipeline[i], Question_Generation_Pipeline[i] , Question_Generation_Pipeline[i]])
  i+=1

    
# name of csv file 
filename = "/content/pipeline.csv"
    
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(rows)

In [54]:
import pandas as pd
df= pd.read_csv('/content/pipeline.csv')

In [55]:
df

,Context_Pipeline,Question_Generation_Pipeline,Answer_Generation_Pipeline
0,"eted, high-level, general-purpose programming language. Created by Guido van...",Who created Python?,Who created Python?
1,programming language. Created by Guido van Rossum and first released in 199...,When was Python first released?,When was Python first released?
2,"by Guido van Rossum and first released in 1991, Python's design philosophy e...",What is Python's design philosophy?,What is Python's design philosophy?
3,Princess Arya Stark is the third child and second daughter of Lord Eddard St...,Who is the third child and second daughter of Lord Eddard Stark and his wife...,Who is the third child and second daughter of Lord Eddard Stark and his wife...
4,"of the incumbent Westerosi monarchs, Sansa, Queen in the North, and Brandon,...",Princess Arya Stark is the sister of what Westerosi monarchs?,Princess Arya Stark is the sister of what Westerosi monarchs?
5,"aughter of Lord Eddard Stark and his wife, Lady Catelyn Stark. She is the si...","What is Sansa, Queen in the North, and Brandon, King of the Andals?","What is Sansa, Queen in the North, and Brandon, King of the Andals?"
6,"persecution of House Stark by House Lannister, Arya is trained as a Faceles...",What is Arya trained as?,What is Arya trained as?
7,"r, Arya is trained as a Faceless Man at the House of Black and White in Braa...",Where is the House of Black and White located?,Where is the House of Black and White located?
8,"ter of the incumbent Westerosi monarchs, Sansa, Queen in the North, and Bran...",What is the name of the first men?,What is the name of the first men?
9,"using her abilities to avenge her family. Upon her return to Westeros, she e...",What is the name of the line that Frey exterminates?,What is the name of the line that Frey exterminates?


## Translated Question Answer Generation Pipeline
Trained models for Question Answer Generation are not available in many languages other than English. Haystack
provides a workaround for that issue by machine-translating a pipeline's inputs and outputs with the
TranslationWrapperPipeline. The following example generates German questions and answers on a German text
document - by using an English model for Question Answer Generation.

In [32]:
# # Fill the document store with a German document.
# text1 = "Python ist eine interpretierte Hochsprachenprogrammiersprache für allgemeine Zwecke. Sie wurde von Guido van Rossum entwickelt und 1991 erstmals veröffentlicht. Die Design-Philosophie von Python legt den Schwerpunkt auf die Lesbarkeit des Codes und die Verwendung von viel Leerraum (Whitespace)."
# docs = [{"content": text1}]
# document_store.delete_documents()
# document_store.write_documents(docs)

# # Load machine translation models
# from haystack.nodes import TransformersTranslator

# in_translator = TransformersTranslator(model_name_or_path="Helsinki-NLP/opus-mt-de-en")
# out_translator = TransformersTranslator(model_name_or_path="Helsinki-NLP/opus-mt-en-de")

# # Wrap the previously defined QuestionAnswerGenerationPipeline
# from haystack.pipelines import TranslationWrapperPipeline

# pipeline_with_translation = TranslationWrapperPipeline(
#     input_translator=in_translator, output_translator=out_translator, pipeline=qag_pipeline
# )

# for idx, document in enumerate(tqdm(document_store)):
#     print(f"\n * Generating questions and answers for document {idx}: {document.content[:100]}...\n")
#     result = pipeline_with_translation.run(documents=[document])
#     print_questions(result)